In [1]:
import pandas as pd
import os

In [2]:
df_raw = pd.read_csv("data/raw_data/20250207 null_sic_dic.csv")
df_raw.head()  

,stay_id,sofa_score,age,los,gender,scr_min,mdrd_est,scr_baseline,alt_min,alp_min,...,bicarbonate_min,bun_max,chloride_max,heart_rate_max,resp_rate_max,sbp_min,dbp_min,mbp_min,sic,dic
0,34216106,2,59,2.127222,1,0.6,1.071318,0.6,NaN,NaN,...,25.0,10,110.0,112.0,29.0,82.5,30.5,55.5,1,0
1,30150318,4,84,3.806713,0,0.9,0.789202,0.9,72.0,40.0,...,19.0,40,106.0,118.0,27.0,92.0,41.0,58.0,1,0
2,36105458,2,53,25.045949,1,0.7,1.093380,0.7,21.0,57.0,...,17.0,14,113.0,112.0,29.0,87.0,54.0,67.0,1,0
3,33668510,3,83,3.078576,1,0.8,1.010712,0.8,NaN,NaN,...,22.0,30,107.0,108.0,37.0,71.0,31.0,49.0,1,0
4,38302034,3,99,2.948507,0,1.5,0.759394,1.5,NaN,NaN,...,23.0,45,102.0,133.0,29.0,88.0,39.0,48.0,1,0


In [7]:
df_raw["dic"].value_counts()

dic
0    10441
1      426
Name: count, dtype: int64

# 缺失值处理

In [4]:
# 先看看有没有字符串类型的数值
string_positions = df_raw.applymap(type).eq(str)

# 获取包含字符串的具体位置（行和列）
positions = string_positions[string_positions].stack()

# 输出位置：行索引和列名
for (row, col), _ in positions.items():
    print(f"字符串出现在行: {row}, 列: {col}")

In [5]:
# 检测缺失比例大于 0.5 的特征（无）
null_feq = df_raw.isnull().mean()>0.5
null_feq.sum()

0

In [6]:
# 统计每行缺失值的数量并删除缺失值大于 5个 的行
df_cleaned = df_raw[df_raw.isnull().sum(axis=1) <= 5]
print(df_raw.shape, df_cleaned.shape)

(10867, 69) (7923, 69)


In [7]:
df_cleaned["dic"].value_counts()

dic
0    7526
1     397
Name: count, dtype: int64

In [15]:
# 设定阈值，比如非重复值个数少于一定数量时认为是分类变量
threshold = 2 # 可根据实际需求调整

# 自动提取列名，根据唯一值数量
columns_by_unique_count = df_cleaned.nunique()  # 每列的唯一值个数
categorical_cols = columns_by_unique_count[columns_by_unique_count <= threshold].index.tolist()
continuous_cols = columns_by_unique_count[columns_by_unique_count > threshold].index.tolist()
print(categorical_cols)
print(continuous_cols)

['gender', 'ventilation', 'vasopressin', 'crrt', 'death', 'myocardial_infarct', 'congestive_heart_failure', 'peripheral_vascular_disease', 'cerebrovascular_disease', 'dementia', 'chronic_pulmonary_disease', 'rheumatic_disease', 'peptic_ulcer_disease', 'mild_liver_disease', 'diabetes_without_cc', 'diabetes_with_cc', 'paraplegia', 'renal_disease', 'malignant_cancer', 'severe_liver_disease', 'metastatic_solid_tumor', 'aids', 'sic', 'dic']
['stay_id', 'sofa_score', 'age', 'los', 'scr_min', 'mdrd_est', 'scr_baseline', 'alt_min', 'alp_min', 'ast_min', 'sirs', 'lods', 'apsiii', 'wbc_max', 'basophils_abs_max', 'eosinophils_abs_min', 'lymphocytes_abs_min', 'monocytes_abs_min', 'neutrophils_abs_max', 'basophils_min', 'eosinophils_max', 'lymphocytes_min', 'monocytes_min', 'neutrophils_max', 'age_score', 'charlson_comorbidity_index', 'albumin_min', 'creatinine_max', 'sodium_min', 'inr_max', 'pt_max', 'ptt_max', 'hemoglobin_min', 'platelet_min', 'rbc_min', 'potassium_max', 'glucose_max', 'bicarbona

In [16]:
df_cleaned[categorical_cols] = df_cleaned[categorical_cols].fillna(0)
df_cleaned[continuous_cols] = df_cleaned.groupby('dic')[continuous_cols].transform(lambda x: x.fillna(x.mean()))

/var/folders/qr/2y03gxbx72v657l4cp_f9lcc0000gp/T/ipykernel_94534/1583608775.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[categorical_cols] = df_cleaned[categorical_cols].fillna(0)
/var/folders/qr/2y03gxbx72v657l4cp_f9lcc0000gp/T/ipykernel_94534/1583608775.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[continuous_cols] = df_cleaned.groupby('dic')[continuous_cols].transform(lambda x: x.fillna(x.mean()))


In [ ]:
#df_cleaned.to_csv("data/清洗后.csv", index=False)

# 分割

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 进行 8:2 的数据集分割，保持 dic 列的比例一致
train_df, test_df = train_test_split(df_cleaned, test_size=0.2, stratify=df_cleaned['dic'], random_state=42)

# 输出结果
print("训练集的大小：", train_df.shape)
print("测试集的大小：", test_df.shape)

# 可以选择保存分割后的数据集
train_df.to_csv("./data/归一前_train_data.csv", index=False)
test_df.to_csv("./data/归一前_test_data.csv", index=False)

训练集的大小： (6338, 69)
验证集的大小： (1585, 69)


# 标准化

In [16]:
from sklearn.preprocessing import StandardScaler

#threshold = 2  # 可根据实际需求调整
#columns_by_unique_count = df_cleaned.nunique()  # 每列的唯一值个数
#categorical_cols = columns_by_unique_count[columns_by_unique_count <= threshold].index.tolist()
#continuous_cols = columns_by_unique_count[columns_by_unique_count > threshold].index.tolist()
print(categorical_cols)
print(continuous_cols)

scaler = StandardScaler()
train_df[continuous_cols] = scaler.fit_transform(train_df[continuous_cols])

# 使用训练集的标准化参数（均值和标准差）来转换测试集
test_df[continuous_cols] = scaler.transform(test_df[continuous_cols])

train_df.to_csv("./data/train_data_standardized.csv", index=False)
test_df.to_csv("./data/test_data_standardized.csv", index=False)

['gender', 'ventilation', 'vasopressin', 'crrt', 'death', 'myocardial_infarct', 'congestive_heart_failure', 'peripheral_vascular_disease', 'cerebrovascular_disease', 'dementia', 'chronic_pulmonary_disease', 'rheumatic_disease', 'peptic_ulcer_disease', 'mild_liver_disease', 'diabetes_without_cc', 'diabetes_with_cc', 'paraplegia', 'renal_disease', 'malignant_cancer', 'severe_liver_disease', 'metastatic_solid_tumor', 'aids', 'sic', 'dic']
['stay_id', 'sofa_score', 'age', 'los', 'scr_min', 'mdrd_est', 'scr_baseline', 'alt_min', 'alp_min', 'ast_min', 'sirs', 'lods', 'apsiii', 'wbc_max', 'basophils_abs_max', 'eosinophils_abs_min', 'lymphocytes_abs_min', 'monocytes_abs_min', 'neutrophils_abs_max', 'basophils_min', 'eosinophils_max', 'lymphocytes_min', 'monocytes_min', 'neutrophils_max', 'age_score', 'charlson_comorbidity_index', 'albumin_min', 'creatinine_max', 'sodium_min', 'inr_max', 'pt_max', 'ptt_max', 'hemoglobin_min', 'platelet_min', 'rbc_min', 'potassium_max', 'glucose_max', 'bicarbona